## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation',
                                              'LSSTCam/runs/nightlyValidation/20250425/w_2025_17/DM-50157'])

In [ ]:
#expId = 2025060100566
expId = 2025053100267
#expId = 2025042500413
instrument='LSSTCam'
where = "exposure.id=expId AND instrument='LSSTCam'"
expRecords = butler.registry.queryDimensionRecords(
            "exposure", where=where, bind={"expId": expId}, datasets="raw"
        )
filteredExpRecords = set(expRecords)
expRecord = filteredExpRecords.pop()
#print(expRecord)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)

In [ ]:
fig = plotMountErrors(mountData, mountErrors)
plt.savefig(f"/home/c/cslage/u/MTMount/mount_plots/Mount_Plot_Hex_{expId}.png")

In [ ]:
from lsst.summit.utils.efdUtils import getEfdData
from astropy.time import Time, TimeDelta
hexData = getEfdData(
    client,
    "lsst.sal.MTHexapod.application",
    begin=Time("2025-04-25T00:00:00"),
    end=Time("2025-04-25T00:10:00")
)
len(hexData)

In [ ]:
azimuthData = getEfdData(
    client,
    "lsst.sal.MTMount.azimuth",
    begin=Time("2025-04-25T00:00:00"),
    end=Time("2025-04-25T00:10:00")
    )
len(azimuthData)

In [ ]:
import copy